# Librerías a instalar

In [1]:
#!pip install -r requirements.txt

# Librerías a importar

In [2]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog as fd
from py_tps import TPSFile, TPSImage, TPSPoints
from PIL import Image, ImageEnhance, ImageStat
from tqdm import tqdm

import random
import os
import csv
import math

# Funciones

## Comprobación de carpetas

In [3]:
def check_folders():
    ori = 'Original/'
    pre = 'PreOutput/'
    col = 'ColorOutput/'
    tps = 'TPS/'
    gro = 'GroundTruth'
    fin = 'FinalOutput/'

    if os.path.exists(ori) is False:
        os.mkdir(ori)
    if os.path.exists(pre) is False:
        os.mkdir(pre)
    if os.path.exists(col) is False:
        os.mkdir(col)
    if os.path.exists(tps) is False:
        os.mkdir(tps)
    if os.path.exists(gro) is False:
        os.mkdir(gro)
    if os.path.exists(fin) is False:
        os.mkdir(fin)

## Seleccionar imagen

In [4]:
def select_image():
    root = tk.Tk()
    root.withdraw()
    file_path = fd.askopenfilename(filetypes=(('Imágenes','*.png'),
                                              ('Imágenes','*.jpg'),
                                              ('Imágenes','*.jpeg')))
    return file_path

## Seleccionar carpeta

In [5]:
def select_folder():
    root = tk.Tk()
    root.withdraw()
    path = fd.askdirectory()
    return path

## Brillo, dimensiones y grupo

In [6]:
def get_brightness(image_path):
    image = Image.open(image_path).convert('L')
    stat = ImageStat.Stat(image)
    return stat.rms[0]

In [7]:
def get_dimensions(image_path):
    image = Image.open(image_path).convert('L')
    width = image.width
    height = image.height
    return width, height

In [8]:
def get_group(image_path):
    image = Image.open(image_path).convert('L')
    width = image.width
    height = image.height
    if width >= 560 and width <= 1286 and height >= 1011 and height <= 1454:
        return 0
    elif width >= 965 and width <= 1533 and height >= 1398 and height <= 1701:
        return 1
    elif width >= 1186 and width <= 1950 and height >= 1635 and height <= 2460:
        return 2
    else:
        return 3

## Pre procesamiento de las imágenes

In [9]:
def pre_processing(file_path):

    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)
    
    image = Image.open(file_path)
    image.save('Original/' + file_name[0] + file_name[1])
    
    #brightness = get_brightness(file_path)
    brightness = ImageStat.Stat(image).rms[0]
    if brightness > 0 and brightness < 25:
        brightness_factor = random.randint(8,10)
    elif brightness >= 25 and brightness < 39:
        brightness_factor = random.randint(3,10)
    elif brightness >= 39 and brightness < 54:
        brightness_factor = random.randint(2,7)
    elif brightness >= 54 and brightness < 72:
        brightness_factor = random.randint(2,6)
    elif brightness >= 72 and brightness < 120:
        brightness_factor = random.randint(2,5)
    else:
        brightness_factor = random.randint(2,10)

    enhancer = ImageEnhance.Brightness(image)
    image_output = enhancer.enhance(brightness_factor)

    cenhancer = ImageEnhance.Contrast(image_output)
    contrast_factor = random.randint(1,4)
    image_output = cenhancer.enhance(contrast_factor)

    output_name = file_name[0] + 'P' + file_name[1]
    image_output.save('PreOutput/' + output_name)

    color_image = cv2.imread('PreOutput/' + output_name,cv2.IMREAD_COLOR)
    color_image = cv2.applyColorMap(color_image, cv2.COLORMAP_MAGMA)
    #HOT AUTUMN SPRING MAGMA

    color_output_name = file_name[0] + 'C' + file_name[1]
    cv2.imwrite('ColorOutput/' + color_output_name, color_image)

In [60]:
def pre_random_processing(file_path):

    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)
    
    image = Image.open(file_path)
    image.save('Original/' + file_name[0] + file_name[1])
    
    #brightness = get_brightness(file_path)
    brightness = ImageStat.Stat(image).rms[0]
    brightness_factor = random.randint(2,5)

    enhancer = ImageEnhance.Brightness(image)
    image_output = enhancer.enhance(brightness_factor)

    cenhancer = ImageEnhance.Contrast(image_output)
    contrast_factor = random.randint(1,5)
    image_output = cenhancer.enhance(contrast_factor)

    output_name = file_name[0] + 'P' + file_name[1]
    image_output.save('PreOutput/' + output_name)

    color_image = cv2.imread('PreOutput/' + output_name,cv2.IMREAD_COLOR)
    color_image = cv2.applyColorMap(color_image, cv2.COLORMAP_MAGMA)
    #HOT AUTUMN SPRING MAGMA

    color_output_name = file_name[0] + 'C' + file_name[1]
    cv2.imwrite('ColorOutput/' + color_output_name, color_image)

In [10]:
def manual_pre_processing(file_path, brightness_factor, contrast_factor):

    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)
    
    image = Image.open(file_path)
    image.save('Original/' + file_name[0] + file_name[1])

    enhancer = ImageEnhance.Brightness(image)
    image_output = enhancer.enhance(brightness_factor)

    cenhancer = ImageEnhance.Contrast(image_output)
    image_output = cenhancer.enhance(contrast_factor)

    output_name = file_name[0] + 'P' + file_name[1]
    image_output.save('PreOutput/' + output_name)

    color_image = cv2.imread('PreOutput/' + output_name,cv2.IMREAD_COLOR)
    color_image = cv2.applyColorMap(color_image, cv2.COLORMAP_MAGMA)
    #HOT AUTUMN SPRING MAGMA

    color_output_name = file_name[0] + 'C' + file_name[1]
    cv2.imwrite('ColorOutput/' + color_output_name, color_image)

## Recorrer archivos

In [11]:
def browse_files(path):
    files = os.listdir(path)
    #for archivo in archivos:
    for file in tqdm(files, desc="Progreso"):
        if (
            file.endswith(".png")
            or file.endswith(".jpg")
            or file.endswith(".jpeg")
        ):
            landmarks_detector(path + "/" + file)

## Ajustar puntos

In [12]:
def fit_landmarks_old(lista, group):
    if group == 0:
        lista[1] = round(5.106151 + (1.012870 * lista[1]))
        lista[2] = round(7.72715 + (1.00686 * lista[2]))
        lista[3] = round(10.789601 + (1.009205 * lista[3]))
        lista[4] = round(9.3004 +(1.0006 * lista[4]))
        lista[9] = round(-0.235306 + (0.998316 * lista[9]))
        lista[10] = round(-4.869106 + (1.021501 * lista[10]))
        lista[11] = round(-7.204788 + (1.009601 * lista[11]))
        lista[12] = round(10.39307 + (1.01130 * lista[12]))
        lista[13] = round(9.06720 + (0.98785 * lista[13]))
        lista[14] = round(2.2351 + (1.0256 * lista[14]))
        lista[14] = round(-1.76335 + (1.00453 * lista[14]))
        lista[19] = round(-14.66464 + (1.02595 * lista[19]))
        lista[20] = round(4.07344 + (1.01691 * lista[20]))
        lista[21] = round(-7.27641 + (1.01007 * lista[21]))
        lista[22] = round(9.18259 + (1.01790 * lista[22]))
        lista[22] = round(-43.3881 + (1.0429 * lista[22]))
        lista[23] = round(16.85348 + (0.95979 * lista[23]))
        lista[24] = round(-22.03858 + (1.05206 * lista[24]))
        lista[29] = round(-11.98862 + (1.02672 * lista[29]))
        lista[30] = round(5.3611 + (1.0145 * lista[30]))
        lista[31] = round(-6.53737 + (1.00182 * lista[31]))
        lista[32] = round(8.18818 + (1.01267 * lista[32]))
        lista[33] = round(22.48798 + (0.92873 * lista[33]))
        lista[34] = round(32.87670 + (0.98157 * lista[34]))
        lista[34] = round(-0.23360 + (1.00043 * lista[34]))
        lista[39] = round(-10.1762 + (1.0079 * lista[39]))
        lista[40] = round(11.743553 + (1.002825 * lista[40]))
        lista[41] = round(-14.43487 + (1.02896 * lista[41]))
        lista[42] = round(10.66727 + (1.00441 * lista[42]))
        lista[43] = round(-30.1293 + (1.0654 * lista[43]))
        lista[44] = round(1.64544 + (1.02445 * lista[44]))
        lista[44] = round(1.39553 + (1.00176 * lista[44]))
    if group == 1:
        lista[1] = math.trunc(3.30588 + (1.01336 * lista[1]))
        lista[2] = math.trunc(3.57418 + (1.01018 * lista[2]))
        lista[3] = math.trunc(-1.82793 + (1.02478 * lista[3]))
        lista[4] = math.trunc(9.99117 + (1.00273 * lista[4]))
        lista[9] = math.trunc(-15.64847 + (1.01073 * lista[9]))
        lista[10] = math.trunc(33.68455 + (0.99284 * lista[10]))
        lista[11] = math.trunc(-10.97392 + (1.00730 * lista[11]))
        lista[12] = math.trunc(25.77068 + (1.00174 * lista[12]))
        lista[13] = math.trunc(-7.80290 + (1.00634 * lista[13]))
        lista[14] = math.trunc(40.17557 + (0.99618 * lista[14]))
        lista[19] = math.trunc(-1.20191 + (0.99126 * lista[19]))
        lista[20] = math.trunc(9.14772 + (1.01379 * lista[20]))
        lista[21] = math.trunc(7.13946 + (0.97326 * lista[21]))
        lista[22] = math.trunc(-10.41647 + (1.03552 * lista[22]))
        lista[23] = math.trunc(-6.66252 + (0.99869 * lista[23]))
        lista[24] = math.trunc(75.99016 + (0.95595 * lista[24]))
        lista[29] = math.trunc(0.70778 + (0.98069 * lista[29]))
        lista[30] = math.trunc(66.63456 + (0.97116 * lista[30]))
        lista[31] = math.trunc(-27.65953 + (1.05134 * lista[31]))
        lista[32] = math.trunc(-12.36347 + (1.03254 * lista[32]))
        lista[33] = math.trunc(-2.18791 + (0.99458 * lista[33]))
        lista[34] = math.trunc(84.23158 + (0.93859 * lista[34]))
        lista[39] = math.trunc(-4.54128 + (0.97621 * lista[39]))
        lista[40] = math.trunc(7.56122 + (1.00985 * lista[40]))
        lista[41] = math.trunc(-26.88801 + (1.06388 * lista[41]))
        lista[42] = math.trunc(42.47644 + (0.97588 * lista[42]))
        lista[43] = math.trunc(-10.07636 + (0.96853 * lista[43]))
        lista[44] = math.trunc(13.98637 + (1.01067 * lista[44]))
    return lista

In [13]:
def fit_landmarks_none(lista, group):
    if group == 0:
        lista[1] = math.trunc(-0.650297 + (1.019520 * lista[1]) - 5)
        lista[2] = math.trunc(-12.16251 + (1.03196 * lista[2]) - 4)
        lista[3] = round(13.98933 + (1.00720 * lista[3]) + 5)
        lista[4] = math.trunc(-12.32562 + (1.03158 * lista[4]) - 5)
        lista[9] = math.trunc(-36.14460 + (1.05004 * lista[9]) - 5)
        lista[10] = math.trunc(-4.048638 + (1.022433 * lista[10]) - 3)
        lista[11] = round(-18.240561 + (1.025446 * lista[11]) - 1)
        lista[12] = round(18.08026 + (1.00668 * lista[12]) + 5)
        lista[13] = math.trunc(-15.29636 + (1.02478 * lista[13]) - 2)
        lista[14] = math.trunc(23.11117 + (1.00503 * lista[14]) + 5)
        lista[19] = math.trunc(-3.64131 + (1.00007 * lista[19]) - 2)
        lista[20] = math.trunc(4.683624 + (1.016374 * lista[20]) + 5)
        lista[21] = math.trunc(15.18390 + (0.96488 * lista[21]) + 5)
        lista[22] = round(38.98645 + (0.99596 * lista[22]) + 5)
        lista[23] = math.trunc(3.71277 + (0.98728 * lista[23]) + 3)
        lista[24] = round(5.95937 + (1.02191 * lista[24]) + 5)
        lista[29] = math.trunc(-10.9714 + (1.0244 * lista[29]) - 2)
        lista[30] = math.trunc(10.216698 + (1.010153 * lista[30]) + 2.5)
        lista[31] = math.trunc(-12.82086 + (1.02450 * lista[31]) - 3)
        lista[32] = round(33.49360 + (0.98994 * lista[32]) + 5)
        lista[33] = math.trunc(-6.67640 + (1.00677 * lista[33]) - 2)
        lista[34] = round(9.78413 + (1.00881 * lista[34]) + 5)
        lista[39] = math.trunc(-8.7454 + (0.9993 * lista[39]) - 3)
        lista[40] = math.trunc(-14.09835 + (1.03086 * lista[40]) - 4)
        lista[41] = round(-9.29355 + (1.00865 * lista[41]) - 3)
        lista[42] = round(-4.9911 + (1.0241 * lista[42]) - 5)
        lista[43] = round(-8.46887 + (0.97513 * lista[43]) - 5)
        lista[44] = math.trunc(-17.7602 + (1.0494 * lista[44]) - 5)

    return lista

In [14]:
def fit_landmarks(lista, group):
    if group == 0:
        lista[1] = round(5.821279 + (1.009297 * lista[1]))
        lista[2] = round(9.793988 + (0.999010 * lista[2]) + 3)
        lista[3] = round(1.564947 + (1.016853 * lista[3]))
        lista[4] = round(-15.8019 + (1.0334 * lista[4]) - 5)
        lista[9] = round(-15.879276 + (1.019681 * lista[9]))
        lista[10] = round(6.496213 + (1.011126 * lista[10]) + 2.5)
        lista[11] = round(-10.028794 + (1.014533 * lista[11]))
        lista[12] = round(5.55470 + (1.01721 * lista[12]) + 5)
        lista[13] = round(-0.575607 + (1.002496 * lista[13]))
        lista[14] = round(-6.169507 + (1.032557 * lista[14]) - 5)
        lista[19] = round(-8.989855 + (1.013064 * lista[19]))
        lista[20] = round(7.72577 + (1.01081 * lista[20]) + 5)
        lista[21] = round(-5.80727 + (1.00449 * lista[21]))
        lista[22] = round(1.90406 + (1.02663 * lista[22]) + 5)
        lista[23] = round(-4.99641 + (1.00262 * lista[23]))
        lista[24] = round(-6.13557 + (1.03023 * lista[24]) - 5)
        lista[29] = round(-2.60331 + (1.00217 * lista[29]))
        lista[30] = round(-6.892532 + (1.025369 * lista[30]) - 5)
        lista[31] = round(-8.72348 + (1.01319 * lista[31]))
        lista[32] = round(1.33042 + (1.01806 * lista[32]) + 7.5)
        lista[33] = round(-2.53319 + (0.99910 * lista[33]))
        lista[34] = round(12.42170 + (1.00411 * lista[34]) + 5)
        lista[39] = round(-9.49488 + (1.01906 * lista[39]))
        lista[40] = round(1.91549 + (1.01331 * lista[40]) + 2.5)
        lista[41] = round(-9.41764 + (1.01086 * lista[41]))
        lista[42] = round(2.644171 + (1.015637 * lista[42]) + 5)
        lista[43] = round(-9.84271 + (0.99006 * lista[43]))
        lista[44] = round(11.390165 + (1.011174 * lista[44]) + 5)
    elif group == 1:
        lista[1] = round(1.141868 + (1.014977 * lista[1]))
        lista[2] = round(11.609755 + (1.001257 * lista[2]) + 2.5)
        lista[3] = round(11.599332 + (1.009687 * lista[3]))
        lista[4] = round(-2.12917 + (1.01677 * lista[4]) - 5)
        lista[9] = round(-12.324926 + (1.010969 * lista[9]))
        lista[10] = round(3.346350 + (1.015339 * lista[10]) + 5)
        lista[11] = round(-3.467802 + (1.001488 * lista[11]))
        lista[12] = round(21.56854 + (1.00631 * lista[12]) + 10)
        lista[13] = round(-5.012186 + (1.004991 * lista[13]))
        lista[14] = round(51.19795 + (0.98165 * lista[14]) + 10)
        lista[19] = round(-6.537249 + (1.002163 * lista[19]))
        lista[20] = round(17.194060 + (1.006092 * lista[20]) + 5)
        lista[21] = round(-8.3329 + (1.0054 * lista[21]))
        lista[22] = round(36.9830 + (0.9944 * lista[22]) + 10)
        lista[23] = round(-3.24230 + (1.00072 * lista[23]))
        lista[24] = round(16.55739 + (1.00748 * lista[24]) + 10)
        lista[29] = round(-4.93771 + (0.99999 * lista[29]))
        lista[30] = round(3.87031 + (1.01495 * lista[30]) + 5)
        lista[31] = round(-8.34573 + (1.01032 * lista[31]))
        lista[32] = round(18.34660 + (1.00646 * lista[32]) + 5)
        lista[33] = round(4.37963 + (0.98682 * lista[33]))
        lista[34] = round(34.87327 + (0.97952 * lista[34]) + 5)
        lista[39] = round(-10.288290 + (1.014644 * lista[39]))
        lista[40] = round(16.48810 + (0.99530 * lista[40]) + 2.5)
        lista[41] = round(-13.821444 + (1.011924 * lista[41]))
        lista[42] = round(7.589769 + (1.008075 * lista[42]) + 2.5)
        lista[43] = round(-18.324325 + (1.008953 * lista[43]))
        lista[44] = round(23.436837 + (0.996013 * lista[44]) + 11.5)
    elif group == 2:
        lista[1] = round(2.555904 + (1.013751 * lista[1]))
        lista[2] = round(-6.313447 + (1.021876 * lista[2]) - 2.5)
        lista[3] = round(21.20420 + (1.00435 * lista[3]))
        lista[4] = round(0.21609 + (1.02271 * lista[4]) + 5)
        lista[9] = round(-16.746214 + (1.014527 * lista[9]))
        lista[10] = round(16.101382 + (1.007505 * lista[10]) + 5)
        lista[11] = round(-29.544946 + (1.029308 * lista[11]))
        lista[12] = round(37.23605 + (0.99969 * lista[12]) + 2.5)
        lista[13] = round(-30.990490 + (1.035389 * lista[13]))
        lista[14] = round(24.242884 + (1.010722 * lista[14]) + 2.5)
        lista[19] = round(-33.50730 + (1.03218 * lista[19]))
        lista[20] = round(23.368763 + (1.009098 * lista[20]) + 5)
        lista[21] = round(-16.79147 + (1.01343 * lista[21]))
        lista[22] = round(35.21449 + (1.00690 * lista[22]) + 5)
        lista[23] = round(-4.403665 + (1.000960 * lista[23]))
        lista[24] = round(44.63241 + (0.99427 * lista[24]) + 10)
        lista[29] = round(-13.2599 + (1.0155 * lista[29]))
        lista[30] = round(4.096145 + (1.018218 * lista[30]) + 7.5)
        lista[31] = round(-8.23720 + (1.00469 * lista[31]))
        lista[32] = round(-0.479272 + (1.023300 * lista[32]) - 5)
        lista[33] = round(7.54141 + (0.98085 * lista[33]))
        lista[34] = round(26.07954 + (0.99318 * lista[34]) + 5)
        lista[39] = round(-11.509642 + (1.003331 * lista[39]))
        lista[40] = round(5.446614 + (1.009385 * lista[40]) + 5)
        lista[41] = round(-8.973508 + (0.993257 * lista[41]))
        lista[42] = round(11.400947 + (1.008980 * lista[42]) + 7.5)
        lista[43] = round(-16.00805 + (0.99553 * lista[43]))
        lista[44] = round(23.519321 + (1.001766 * lista[44]) + 7.5)
    else:
        lista[1] = round(17.429752 + (1.003707 * lista[1]))
        lista[2] = round(0.882041 + (1.013096 * lista[2]) + 2.5)
        lista[3] = round(25.7573 + (0.9986 * lista[3]))
        lista[4] = round(25.684338 + (0.999939 * lista[4]) + 5)
        lista[9] = round(-11.273166 + (1.003843 * lista[9]))
        lista[10] = round(14.997216 + (1.007823 * lista[10]) + 5)
        lista[11] = round(-1.924864 + (1.000091 * lista[11]))
        lista[12] = round(23.620020 + (1.003821 * lista[12]) + 5)
        lista[13] = round(-2.163268 + (1.003495 * lista[13]))
        lista[14] = round(12.905160 + (1.004440 * lista[14]) + 5)
        lista[19] = round(-1.53967 + (0.99731 * lista[19]))
        lista[20] = round(29.742991 + (0.999150 * lista[20]) + 10)
        lista[21] = round(-0.138459 + (0.998830 * lista[21]))
        lista[22] = round(13.498307 + (1.005544 * lista[22]) + 10)
        lista[23] = round(2.249794 + (0.992409 * lista[23]))
        lista[24] = round(-3.98567 + (1.01105 * lista[24]) - 10)
        lista[29] = round(-4.058299 + (0.998010 * lista[29]))
        lista[30] = round(21.050639 + (1.000977 * lista[30]) + 10)
        lista[31] = round(-8.055530 + (1.006464 * lista[31]))
        lista[32] = round(15.080651 + (1.003490 * lista[32]) + 10)
        lista[33] = round(-9.100602 + (1.003799 * lista[33]))
        lista[34] = round(-10.229015 + (1.011111 * lista[34]) - 10)
        lista[39] = round(1.290823 + (0.983048 * lista[39]))
        lista[40] = round(-4.439931 + (1.011739 * lista[40]) - 10)
        lista[41] = round(1.54911 + (0.98240 * lista[41]))
        lista[42] = round(-8.80657 + (1.01357 * lista[42]) - 10)
        lista[43] = round(-3.207830 + (1.003308 * lista[43]))
        lista[44] = round(-10.197507 + (1.017120 * lista[44]) - 10)
    return lista

## Ajustar puntos de Mediapipe

In [15]:
def fit_mediapipe(hand_landmarks, lista, width, height):
    hand_landmarks.landmark[4].x = lista[1]/width
    hand_landmarks.landmark[4].y = 1 - lista[2]/height
    hand_landmarks.landmark[3].x = lista[3]/width
    hand_landmarks.landmark[3].y = 1 - lista[4]/height
    hand_landmarks.landmark[8].x = lista[9]/width
    hand_landmarks.landmark[8].y = 1 - lista[10]/height
    hand_landmarks.landmark[7].x = lista[11]/width
    hand_landmarks.landmark[7].y = 1 - lista[12]/height
    hand_landmarks.landmark[6].x = lista[13]/width
    hand_landmarks.landmark[6].y = 1 - lista[14]/height
    hand_landmarks.landmark[12].x = lista[19]/width
    hand_landmarks.landmark[12].y = 1 - lista[20]/height
    hand_landmarks.landmark[11].x = lista[21]/width
    hand_landmarks.landmark[11].y = 1 - lista[22]/height
    hand_landmarks.landmark[10].x = lista[23]/width
    hand_landmarks.landmark[10].y = 1 - lista[24]/height
    hand_landmarks.landmark[16].x = lista[29]/width
    hand_landmarks.landmark[16].y = 1 - lista[30]/height
    hand_landmarks.landmark[15].x = lista[31]/width
    hand_landmarks.landmark[15].y = 1 - lista[32]/height
    hand_landmarks.landmark[14].x = lista[33]/width
    hand_landmarks.landmark[14].y = 1 - lista[34]/height
    hand_landmarks.landmark[20].x = lista[39]/width
    hand_landmarks.landmark[20].y = 1 - lista[40]/height
    hand_landmarks.landmark[19].x = lista[41]/width
    hand_landmarks.landmark[19].y = 1 - lista[42]/height
    hand_landmarks.landmark[18].x = lista[43]/width
    hand_landmarks.landmark[18].y = 1 - lista[44]/height
    
    return hand_landmarks

## Guardar TPS

In [16]:
def save_tps(array, file_name):
    points = TPSPoints(array)
    image = TPSImage(file_name[0]+file_name[1],landmarks=points)
    tps_file = TPSFile([image])
    tps_file.write_to_file('TPS/' + file_name[0] + '.TPS')

## Guardar CSV

In [17]:
def save_csv(lista):
    file = "Puntos.csv"

    if os.path.isfile(file) is False:
        with open("Puntos.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(
                [
                    "ID",
                    "X1",
                    "Y1",
                    "X2",
                    "Y2",
                    "X3",
                    "Y3",
                    "X4",
                    "Y4",
                    "X5",
                    "Y5",
                    "X6",
                    "Y6",
                    "X7",
                    "Y7",
                    "X8",
                    "Y8",
                    "X9",
                    "Y9",
                    "X10",
                    "Y10",
                    "X11",
                    "Y11",
                    "X12",
                    "Y12",
                    "X13",
                    "Y13",
                    "X14",
                    "Y14",
                    "X15",
                    "Y15",
                    "X16",
                    "Y16",
                    "X17",
                    "Y17",
                    "X18",
                    "Y18",
                    "X19",
                    "Y19",
                    "X20",
                    "Y20",
                    "X21",
                    "Y21",
                    "X22",
                    "Y22",
                    "X23",
                    "Y23",
                    "X24",
                    "Y24",
                    "X25",
                    "Y25",
                    "X26",
                    "Y26",
                    "X27",
                    "Y27",
                    "X28",
                    "Y28",
                ]
            )
            csvfile.close()

    with open("Puntos.csv", "a", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(lista)
        csvfile.close()

## Ordenar CSV

In [18]:
def sort_csv(file):
    df = pd.read_csv(file)
    df = df.sort_values('ID')
    df.to_csv(file, index=False)

In [19]:
def fit_drawing(lista, height):
    lista[2] = int(height * (1 - lista[2]/height))
    lista[4] = int(height * (1 - lista[4]/height))
    lista[10] = int(height * (1 - lista[10]/height))
    lista[12] = int(height * (1 - lista[12]/height))
    lista[14] = int(height * (1 - lista[14]/height))
    lista[20] = int(height * (1 - lista[20]/height))
    lista[22] = int(height * (1 - lista[22]/height))
    lista[24] = int(height * (1 - lista[24]/height))
    lista[30] = int(height * (1 - lista[30]/height))
    lista[32] = int(height * (1 - lista[32]/height))
    lista[34] = int(height * (1 - lista[34]/height))
    lista[40] = int(height * (1 - lista[40]/height))
    lista[42] = int(height * (1 - lista[42]/height))
    lista[44] = int(height * (1 - lista[44]/height))

    return lista

In [20]:
def draw_sarai_landmarks(file_path):
    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)
    original = cv2.imread("Original/" + file_name[0] + file_name[1])
    sarai = pd.read_csv("GroundTruth.csv")
    lista_sarai = sarai.loc[sarai['ID'] == int(file_name[0])].values.flatten().tolist()
    width, height = get_dimensions(file_path)
    hand_landmarks_sarai = fit_drawing(lista_sarai, height)
    
    index = [1, 2, 3, 4, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24, 29, 
        30, 31, 32, 33, 34, 39, 40, 41, 42, 43, 44]
    
    x = 0
    while x < len(index):
        y = x + 1
        center_coordinates = (hand_landmarks_sarai[index[x]], hand_landmarks_sarai[index[y]])
        radius = 5
        color = (0, 255, 0) # BGR
        thickness = -1
        original = cv2.circle(original, center_coordinates, radius, color, thickness)
        
        x = x + 2
    
    cv2.imwrite("GroundTruth/" + file_name[0] + file_name[1], original)

In [25]:
def draw_landmarks(file_path, lista,g):
    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)
    original = cv2.imread("GroundTruth/" + file_name[0] + file_name[1])
    width, height = get_dimensions(file_path)
    lista = fit_drawing(lista, height)
    
    index = [1, 2, 3, 4, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24, 29, 
        30, 31, 32, 33, 34, 39, 40, 41, 42, 43, 44]
    
    x = 0
    while x < len(index):
        y = x + 1
        center_coordinates = (lista[index[x]], lista[index[y]])
        radius = 5
        color = (0, 0, 255) # BGR
        thickness = -1
        original = cv2.circle(original, 
                              center_coordinates, 
                              radius, 
                              color, 
                              thickness)
        
        x = x + 2
    #if g == 0:
        #cv2.imwrite("FinalOutput/G0/" + file_name[0] + "F" + file_name[1], original)
    #elif g == 1:
        #cv2.imwrite("FinalOutput/G1/" + file_name[0] + "F" + file_name[1], original)
    #elif g == 2:
        #cv2.imwrite("FinalOutput/G2/" + file_name[0] + "F" + file_name[1], original)
    #else:
        #cv2.imwrite("FinalOutput/G3/" + file_name[0] + "F" + file_name[1], original)
    cv2.imwrite("FinalOutput/" + file_name[0] + "F" + file_name[1], original)

## Localizar puntos

In [79]:
def landmarks_detector(file_path):
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    mp_drawing_styles = mp.solutions.drawing_styles
    
    file_name = os.path.basename(file_path)
    file_name = os.path.splitext(file_name)
    lista = [file_name[0]]
    draw_sarai_landmarks(file_path)
    
    intentos = 1
    bandera = True

    while bandera:
        if intentos < 50:
            pre_processing(file_path)
        else:
            pre_random_processing(file_path)
        #manual_pre_processing(file_path,1,4)

        image = cv2.imread("ColorOutput/" + file_name[0] + "C" + file_name[1])

        umbral = 0.9
        alerta = False
        while True:
            with mp_hands.Hands(
                static_image_mode=True, max_num_hands=2, min_detection_confidence=umbral
            ) as hands:
                height, width, _ = image.shape
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image_rgb)
                if results.multi_hand_landmarks is not None:
                    index = [
                        4,
                        3,
                        2,
                        1,
                        8,
                        7,
                        6,
                        5,
                        12,
                        11,
                        10,
                        9,
                        16,
                        15,
                        14,
                        13,
                        20,
                        19,
                        18,
                        17,
                    ]
                    for hand_landmarks in results.multi_hand_landmarks:
                        for i in index:
                            x = round(hand_landmarks.landmark[i].x * width)
                            y = height - round(hand_landmarks.landmark[i].y * height)
                            lista.append(x)
                            lista.append(y)
                            if i == 5 or i == 9 or i == 13:
                                lista.append(0)
                                lista.append(0)
                            if i == 6:
                                mediana = height - round(hand_landmarks.landmark[4].y * height)
                                if y < mediana:
                                    alerta = True

                    for x in range(5):
                        lista.append(0)
                        lista.append(0)

            if alerta == True:
                print(file_name[0])
                results.multi_handedness = None
                lista = [file_name[0]]
            if results.multi_handedness is None:
                umbral = umbral - 0.1
            if umbral < 0.3:
                intentos = intentos + 1
                if intentos == 70:
                    #print("Imagen: " + file_name[0] + " None")
                    for k in range(56):
                        lista.append(0)
                    save_csv(lista)
                    bandera = False
                break
            if results.multi_handedness != None:
                # print(
                #     "Imagen: " + file_name[0] + "Done. Intentos:",
                #     intentos,
                #     "Umbral:",
                #     umbral,
                # )
                g = get_group(file_path)
                adjusted_list = fit_landmarks(lista, g)
                #adjusted_list = lista
                height, width, _ = image.shape
                save_csv(adjusted_list)
                save_tps(np.array(adjusted_list[1:]).reshape(28,2), file_name)
                draw_landmarks(file_path, adjusted_list,g)
                #mp_drawing.draw_landmarks(original, hand_landmarks) # mp_hands.HAND_CONNECTIONS
                #cv2.imwrite("FinalOutput/" + file_name[0] + "F" + file_name[1], original)

                bandera = False
                break

# Prueba seleccionando una carpeta

In [80]:
#check_folders()
path = select_folder()
browse_files(path)
sort_csv('Puntos.csv')

Progreso:  42%|████▏     | 67/161 [02:28<03:02,  1.94s/it]

15226
15226
15226
15226
15226
15226
15226


Progreso:  73%|███████▎  | 117/161 [08:45<01:34,  2.16s/it]

5672
5672
5672
5672
5672
5672


Progreso: 100%|██████████| 161/161 [10:12<00:00,  3.81s/it]


# Prueba seleccionando una imagen

In [76]:
#check_folders()
image_path = select_image()
landmarks_detector(image_path)
# sort_csv('Puntos.csv')